In [ ]:
import awkward as ak
import dask_awkward as dak
from coffea import processor
import hist
import hist.dask as hda
import numpy as np

########################################################################################################################
# Define the Control Plot Processor object
########################################################################################################################

class ControlPlotProcessor(processor.ProcessorABC):
    """
    This processor creates N-D histograms.
    """

    def __init__(self, var_list, bin_edges_list, channel):
        # Define parameters of processor
        self.var_list = var_list
        self.bin_edges_list = bin_edges_list
        self.channel = channel
        self.axis_labels = self.var_list
        
        pass
        

        
    def process(self, events):
        ################################################################################################################
        # Preprocess dataset
        events = selection(events, self.channel)

        ################################################################################################################
        # Grab the weights - not yet fully implemented
        weights = dak.ones_like(events.MET.pt)
        # EWK Corrections
        # weights *= electroweak_corrections(events, weights, self.rochester_file)

        ################################################################################################################
        # Compute any external variables and collect variables from events
        fill_variables = []
        for var in self.var_list:
            if (var not in events.fields): # check if var needs to be computed
                fill_variables.append(calc_external_var(var, events))
            else:
                fill_variables.append(events[var])
             
        ################################################################################################################
        # Create the histogram object
        axes = [hist.axis.Variable(self.bin_edges_list[j], name=self.axis_labels[j])
                for j in range(len(self.bin_edges_list))]
        histogram = hda.Hist(*axes, storage=hist.storage.Weight()) #hda - dask compatible histogram package

        ################################################################################################################
        # Fill the histogram
        histogram.fill(*fill_variables, weight = weights)
        
        ################################################################################################################
        # Correcting the weights to sqrt(N) from sqrt(sum(weights^2))
        histogram[...] = (np.concatenate((histogram.values()[..., None], 
                                                    histogram.variances()[..., None]), 
                                                    axis=-1)
                                                    )
        return({"histogram": histogram})


    def postprocess(self, accumulator):
        pass
